In [47]:
# Step 1: Install Libraries
!pip install langgraph langchain_groq langchain langchain-community faiss-cpu pypdf

In [48]:
import os
groq_api_key = os.getenv("GROQ_API_KEY")


In [49]:
# Step 2: Import libraries

import os
from typing import TypedDict, Dict
from langgraph.graph import StateGraph, END
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [50]:
# Step 3: Setup LLM

from google.colab import userdata
groq_api_key=userdata.get("GROQ_API_KEY")

In [51]:
# Test the LLM

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Say hello in a friendly way.")
chain = prompt | llm
print(chain.invoke({}).content)


Hello there, how are you doing today? It's great to connect with you.


In [52]:
# Step 4: Define Chatbot State

from typing import TypedDict

class State(TypedDict):
    query: str
    category: str
    sentiment: str
    response: str

In [53]:
# Step 5: Helper to run prompts through the LLM
# ----------------------------
def run_prompt(template: str, query: str) -> str:
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | llm
    # .invoke returns an object with .content — we strip whitespace
    return chain.invoke({"query": query}).content.strip()


In [54]:
# Step 6: categorization (returns descriptive category text)
# ----------------------------
def categorize(state: State) -> Dict[str, str]:
    template = (
        "Categorize this customer query into one of these categories: "
        "Billing, Technical, or General. "
        "Include the category word clearly in your answer and a short explanation.\n\n"
        "Query: {query}"
    )
    category_text = run_prompt(template, state["query"])
    return {"category": category_text}


In [55]:
# Step 7: sentiment analysis (returns descriptive sentiment text)
# ----------------------------
def analyze_sentiment(state: State) -> Dict[str, str]:
    template = (
        "Analyze the sentiment of this customer query. Your answer should contain one of: "
        "Positive, Neutral, or Negative, and you may add a brief reason.\n\n"
        "Query: {query}"
    )
    sentiment_text = run_prompt(template, state["query"])
    return {"sentiment": sentiment_text}


In [56]:
# Step 8: Routing: reads descriptive outputs and decides next handler

def route_query(state: State) -> str:
    sent = state["sentiment"].lower()
    cat = state["category"].lower()

    # Escalate if sentiment indicates negative
    if "negative" in sent:
        return "escalate"

    # Otherwise decide by category keywords
    if "technical" in cat:
        return "technical"
    if "billing" in cat:
        return "billing"
    # default fallback
    return "general"


# Handlers: produce final responses

def handle_technical(state: State) -> Dict[str, str]:
    template = "You are a friendly technical support agent. Provide a clear troubleshooting reply for: {query}"
    resp = run_prompt(template, state["query"])
    return {"response": resp}

def handle_billing(state: State) -> Dict[str, str]:
    template = "You are a friendly billing support agent. Provide a clear billing-related reply for: {query}"
    resp = run_prompt(template, state["query"])
    return {"response": resp}

def handle_general(state: State) -> Dict[str, str]:
    template = "You are a helpful support agent. Provide a concise general reply for: {query}"
    resp = run_prompt(template, state["query"])
    return {"response": resp}

def escalate(state: State) -> Dict[str, str]:
    # static escalation message (you can replace contact details)
    return {"response": "This query has been escalated to a human agent due to negative sentiment. A human will contact you shortly."}


# Orchestrator: runs the full flow using State

def process_query(query: str) -> State:
    state: State = {"query": query, "category": "", "sentiment": "", "response": ""}

    # categorization
    state.update(categorize(state))

    # sentiment
    state.update(analyze_sentiment(state))

    # routing
    route = route_query(state)

    # handle according to route
    if route == "escalate":
        state.update(escalate(state))
    elif route == "technical":
        state.update(handle_technical(state))
    elif route == "billing":
        state.update(handle_billing(state))
    else:
        state.update(handle_general(state))

    return state

In [57]:
# Test the chatbot with sample queries
# ----------------------------
if __name__ == "__main__":
    examples = [
        "I was double charged on my last bill.",
        "My internet is not working properly and keeps disconnecting.",
        "What are your business hours and where are you located?",
        "Your service is so frustrating, nothing works and I'm angry!"
    ]

    for q in examples:
        result = process_query(q)
        print("User Query:", result["query"])
        print("Category (LLM):", result["category"])
        print("Sentiment (LLM):", result["sentiment"])
        print("Bot Response:", result["response"])
        print("-" * 70)


User Query: I was double charged on my last bill.
Category (LLM): **Billing**: The customer query is about a billing issue, specifically a double charge on their last bill.
Sentiment (LLM): Negative

The customer's query expresses a problem they're experiencing (double charging) which is likely to cause frustration and annoyance, leading to a negative sentiment.
Bot Response: This query has been escalated to a human agent due to negative sentiment. A human will contact you shortly.
----------------------------------------------------------------------
User Query: My internet is not working properly and keeps disconnecting.
Category (LLM): **Technical**

This query is categorized as Technical because it pertains to a specific issue with a service (internet connectivity) rather than billing or general information. The customer is seeking assistance with resolving a technical problem with their internet connection.
Sentiment (LLM): Negative

The customer is expressing a problem with their